<span style='font-size: 28pt;'><i>Kaggle - Titanic</i></span>

In [201]:
import numpy as np
import pandas as pd
import pylab as pl
import matplotlib.pyplot as plt
import seaborn as sns


plt.style.use('ggplot')`
%matplotlib inline

## Считывание данных

In [202]:
df = pd.read_csv(r'train.csv')
test = pd.read_csv(r'test.csv')
display(df)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


## Предпроцессинг данных

### Удаление бессмысленных колонок

In [203]:
# Имена и id не имеют значения
df.drop(columns=['Name', 'PassengerId', 'Ticket', 'Cabin'], inplace=True)
display(df)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


### Обработка пропущенных значений
- Замена пропущенных значений на среднее в столбце Age
- Удаление строк с пропущенными значениями в столбце Embarked
- Замена 'male', 'female' на '1', '0' соответственно в столбце Sex 

In [ ]:
df['Age'].fillna(df['Age'].mean(), inplace=True)
#df['Embarked'].fillna(df['Embarked'].mode(), inplace=True)
df['Embarked'].dropna(inplace=True)
df['Sex'].replace('male', 1, inplace=True)
df['Sex'].replace('female', 0, inplace=True)

### Даммирование данных

In [204]:
dummy = pd.get_dummies(df)
display(dummy)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,3,1,22.000000,1,0,7.2500,0,0,1
1,1,1,0,38.000000,1,0,71.2833,1,0,0
2,1,3,0,26.000000,0,0,7.9250,0,0,1
3,1,1,0,35.000000,1,0,53.1000,0,0,1
4,0,3,1,35.000000,0,0,8.0500,0,0,1
...,...,...,...,...,...,...,...,...,...,...
886,0,2,1,27.000000,0,0,13.0000,0,0,1
887,1,1,0,19.000000,0,0,30.0000,0,0,1
888,0,3,0,29.699118,1,2,23.4500,0,0,1
889,1,1,1,26.000000,0,0,30.0000,1,0,0


In [205]:
x = dummy.loc[::,'Pclass':].values
y = dummy['Survived'].values
x.shape, y.shape

((891, 9), (891,))

## Обучение дерева решений

### Разбиение выборки на тестовую и тренировочную части

In [206]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

### Поиск самого точного дерева решений

In [207]:
from sklearn.tree import DecisionTreeClassifier


data = pd.DataFrame(columns=['max_depth', 'criterion', 'min_samples_leaf', 'train_result', 'test_result'])
for criterion in ['entropy']:
    for max_depth in range(5, 100):
        for min_leaf in range(1, 100):
            tree = DecisionTreeClassifier(max_depth=max_depth, random_state=0, criterion=criterion, min_samples_leaf=min_leaf)
            tree.fit(X_train, y_train)
            train_result = tree.score(X_train, y_train)
            test_result = tree.score(X_test, y_test)
            data = data.append({'max_depth': max_depth,
                         'criterion': criterion, 
                         'min_samples_leaf': min_leaf,
                         'train_result': train_result,
                         'test_result': test_result}, ignore_index=True)
display(data)

,max_depth,criterion,min_samples_leaf,train_result,test_result
0,5,entropy,1,0.863563,0.802239
1,5,entropy,2,0.860353,0.802239
2,5,entropy,3,0.858748,0.802239
3,5,entropy,4,0.857143,0.813433
4,5,entropy,5,0.857143,0.813433
...,...,...,...,...,...
9400,99,entropy,95,0.797753,0.761194
9401,99,entropy,96,0.797753,0.761194
9402,99,entropy,97,0.797753,0.761194
9403,99,entropy,98,0.797753,0.761194


### Получение параметров самого точного дерева

In [208]:
index = np.argmax(np.array(data['test_result']))
depth, criterion, leafs = data.iloc[index][:3]
print('Наилучшие параметры Решающего дерева:' + '\n  max_depth:', depth, '\n  criterion:', criterion, '\n  min_samples_leaf:', leafs)
print()

tree=DecisionTreeClassifier(criterion=criterion, max_depth=depth, min_samples_leaf=leafs)
tree.fit(X_train, y_train)
print('Результат на обучающей выборке: ', tree.score(X_train,y_train))
print('Результат на тестовой выборке: ',tree.score(X_test, y_test))

Наилучшие параметры Решающего дерева:
  max_depth: 8 
  criterion: entropy 
  min_samples_leaf: 8

Результат на обучающей выборке:  0.8747993579454254
Результат на тестовой выборке:  0.8470149253731343


## Подготовка тестовой выборки к проверке

### Применение преобразований аналогичных тестовой выборке  

In [ ]:
test.drop(columns=['Name', 'Ticket', 'Cabin'], inplace=True)
display(test)

In [210]:
test['Age'].fillna(test['Age'].mean(), inplace=True)
#test['Embarked'].fillna(test['Embarked'].mode(), inplace=True)
test['Embarked'].dropna(inplace=True)
test['Sex'].replace('male', 1, inplace=True)
test['Sex'].replace('female', 0, inplace=True)
test['Fare'].fillna(test['Fare'].median(), inplace=True)
test_d = pd.get_dummies(test.iloc[:, 1:])
display(test_d)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,3,1,34.50000,0,0,7.8292,0,1,0
1,3,0,47.00000,1,0,7.0000,0,0,1
2,2,1,62.00000,0,0,9.6875,0,1,0
3,3,1,27.00000,0,0,8.6625,0,0,1
4,3,0,22.00000,1,1,12.2875,0,0,1
...,...,...,...,...,...,...,...,...,...
413,3,1,30.27259,0,0,8.0500,0,0,1
414,1,0,39.00000,0,0,108.9000,1,0,0
415,3,1,38.50000,0,0,7.2500,0,0,1
416,3,1,30.27259,0,0,8.0500,0,0,1


## Вывод прогноза дерева в csv таблицу

In [211]:
result = tree.predict(test_d)
answer_df = pd.DataFrame(columns=['PassengerId', 'Survived'])
answer_df['PassengerId'] = test['PassengerId']
answer_df['Survived'] = result
answer_df.to_csv(r'result.csv', index=False)
display(answer_df)

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
